In [1]:
import os, sys
sys.path.append('../src/')

from copulae1 import *

import json
with open('../src/configs/BBT_future_Tiingo_xrp_MM.json','r') as file:
    config = json.load(file)
config
data_name = config['data_name']
calibration_method  = config['calibration_method']
OHR = pd.read_hdf('../results/'+data_name +'/'+calibration_method+'/best_h.h5')
if not os.path.exists("../results/" + data_name + "/MLE/reports"):
	print("Create new folder for results")
	os.mkdir("../results/" + data_name + "/MLE/reports")
	os.mkdir("../results/" + data_name + "/MM/reports")
    
if calibration_method == "MLE":
    report_path = "../results/" + data_name + "/MLE/reports/"
    result_path = "../results/" + data_name + "/MLE/"
elif calibration_method == "MM":
    report_path = "../results/" + data_name + "/MM/reports/"
    result_path = "../results/" + data_name + "/MM/"

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Create new folder for results


In [2]:
OHR

97.csv                                                      \
                 Variance  ERM k=10 ES q=0.01 ES q=0.05 VaR q=0.01 VaR q=0.05   
Gaussian         1.165820  1.054199  1.065234  1.075293   1.081250   1.044336   
t_Copula         1.162891  1.050293  1.058789  1.068457   1.109863   1.062500   
t_Copula_Capped  1.162891  1.050293  1.058789  1.068457   1.109863   1.062500   
Clayton          0.967969  0.811230  0.609277  0.758203   0.713086   0.921680   
Frank            1.162207  1.084375  0.925977  1.048633   1.067285   1.166016   
Gumbel           1.173926  1.209766  1.295410  1.263965   1.319531   1.197070   
Plackett         1.106641  1.014160  0.774512  0.957812   0.943066   1.071875   
Gauss Mix Indep  1.100977  1.010059  0.619531  0.973340   0.887305   1.090820   

                   83.csv                                ...    65.csv  \
                 Variance  ERM k=10 ES q=0.01 ES q=0.05  ... ES q=0.01   
Gaussian         0.896191  0.860059  0.884766  0.888184  ...  0.779590   
t_Copula         0.893262  0.855469  0.879492  0.882910  ...  0.808887   
t_Copula_Capped  0.902539  0.864551  0.858203  0.884473  ...  0.808887   
Clayton          0.749707  0.655371  0.518066  0.617773  ...  0.466895   
Frank            0.878613  0.864453  0.783594  0.846777  ...  0.711816   
Gumbel           0.884473  0.975000  1.056152  1.057422  ...  0.923340   
Plackett         0.842969  0.824609  0.683203  0.788379  ...  0.627832   
Gauss Mix Indep  0.780762  0.759473  0.332324  0.621484  ...  0.376562   

                                                   71.csv                      \
                ES q=0.05 VaR q=0.01 VaR q=0.05  Variance  ERM k=10 ES q=0.01   
Gaussian         0.732227   0.765527   0.669922  0.743359  0.737891  0.771094   
t_Copula         0.772852   0.791992   0.719727  0.768750  0.764062  0.785547   
t_Copula_Capped  0.772852   0.791992   0.719727  0.768750  0.764062  0.785547   
Clayton          0.515820   0.510156   0.598242  0.652051  0.574316  0.466406   
Frank            0.695117   0.703223   0.656738  0.741504  0.738672  0.701074   
Gumbel           0.846387   0.925098   0.756250  0.728711  0.836133  0.941211   
Plackett         0.648145   0.646094   0.653809  0.710059  0.709863  0.613477   
Gauss Mix Indep  0.514941   0.451953   0.731055  0.636426  0.649121  0.328027   

                                                 
                ES q=0.05 VaR q=0.01 VaR q=0.05  
Gaussian         0.760547   0.782422   0.749609  
t_Copula         0.780762   0.793652   0.753711  
t_Copula_Capped  0.780762   0.793652   0.753711  
Clayton          0.540234   0.511523   0.659277  
Frank            0.726074   0.730273   0.746875  
Gumbel           0.901855   0.964551   0.840234  
Plackett         0.683398   0.650000   0.744043  
Gauss Mix Indep  0.523047   0.423437   0.750586  

[8 rows x 672 columns]

In [370]:
natp_path = '../_mathematica/data/'
natp_ls = os.listdir(natp_path)
natp_h = [l for l in natp_ls if l.endswith('_h.csv')]
file_names = [l.replace('_h','') for l in natp_h]
OHR_combined = []
for i in range(len(natp_h)):
    _natp_h = pd.DataFrame(open(natp_path+natp_h[i], 'r').readlines())
    _natp_h = _natp_h.iloc[1:7,:]
    file_name = natp_h[i].replace('_h','')
    for i in range(len(_natp_h)):
        _natp_h.iloc[i,0] = np.float32(_natp_h.iloc[i,0].replace('\n',''))
    _natp_h.columns = ['NIG_factor']
    _natp_h.index = ['Variance', 'VaR q=0.01', 'VaR q=0.05', 'ES q=0.01', 'ES q=0.05', 'ERM k=10']
    _natp_h = _natp_h.T
    
    OHR_combined.append(OHR.loc[:,file_name].append(_natp_h))
OHR = pd.concat(OHR_combined, axis=1, keys=file_names).dropna(axis=1)

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
# Copula_names = ['Gaussian', 't_Copula', 'Clayton', 'Frank', 'Gumbel', 'Plackett', 'Gauss Mix Indep', 'NIG_factor']
Copula_names = ['Gaussian', 't_Copula', 'Clayton', 'Frank', 'Gumbel', 'Plackett', 'Gauss Mix Indep']

OHR = OHR.loc[Copula_names,:]

In [4]:
temp = []
for C in Copula_names:
    hs_by_Copula = OHR.loc[C:C,:].melt()
    hs_by_Copula.columns = ['file_name', 'risk_measure', 'OHR']
    hs_by_Copula.loc[:, 'copula'] = C
    temp.append(hs_by_Copula)

OHR_to_math = pd.concat(temp, axis=0)

In [5]:
v5_path = '../processed_data/'+config['data_name'].replace('3', '5')+'/test/'
v3_path = '../processed_data/'+config['data_name']+'/test/'

v5_files = os.listdir(v5_path)
v3_files = os.listdir(v3_path)

In [6]:
pd.read_csv(v5_path+'0.csv').tail()

,Unnamed: 0,Date,CRIX,future,log return CRIX,log return future
95,95,2020-09-18 00:00:00+00:00,31525.169649,10975.0,0.002718,-0.006358
96,96,2020-09-17 00:00:00+00:00,31439.598392,11045.0,0.014015,0.014133
97,97,2020-09-16 00:00:00+00:00,31002.050083,10890.0,0.005421,0.012939
98,98,2020-09-15 00:00:00+00:00,30834.443871,10750.0,0.026179,0.041307
99,99,2020-09-14 00:00:00+00:00,30037.716520,10315.0,0.001040,-0.005318


In [7]:
pd.read_csv(v3_path+'19.csv') 

,Unnamed: 0,Date,CRIX,future,log return CRIX,log return future
0,95,2020-09-18 00:00:00+00:00,31525.169649,10975.0,0.002718,-0.006358
1,96,2020-09-17 00:00:00+00:00,31439.598392,11045.0,0.014015,0.014133
2,97,2020-09-16 00:00:00+00:00,31002.050083,10890.0,0.005421,0.012939
3,98,2020-09-15 00:00:00+00:00,30834.443871,10750.0,0.026179,0.041307
4,99,2020-09-14 00:00:00+00:00,30037.716520,10315.0,0.001040,-0.005318


In [8]:
OHR_to_use = OHR_to_math.copy()
OHR_to_use.file_name = OHR_to_use.file_name.apply(lambda x: int(x.replace('.csv',''))-19) # shift file names
OHR_to_use = OHR_to_use.loc[OHR_to_use.file_name >0,:] # remove redudant rows
OHR_to_use.file_name = OHR_to_use.file_name.apply(lambda x: str(x)+'.csv')

In [9]:
risk_measure = list(OHR_to_use.risk_measure.unique())
ls = list(OHR_to_use.file_name.unique())

In [10]:
OHR_to_use.loc[:,"HE"] = 0 # Add a new column for results of HE

In [11]:
OHR_to_use.head()

,file_name,risk_measure,OHR,copula,HE
78,1.csv,Variance,0.895117,Gaussian,0
79,1.csv,ERM k=10,0.875879,Gaussian,0
80,1.csv,ES q=0.01,0.989648,Gaussian,0
81,1.csv,ES q=0.05,0.898340,Gaussian,0
82,1.csv,VaR q=0.01,0.924609,Gaussian,0


In [12]:
def hedging_effectiveness2(rm, rs, rf, h):
    rh = rs - h*rf
    if rm == 'ERM k=10':
        return(1-ERM_estimate_trapezoidal(10, rh)/ERM_estimate_trapezoidal(10, rs))

    elif rm == 'ES q=0.01':
        return(1-ES(0.01, rh)/ES(0.01, rs))

    elif rm == 'ES q=0.05':
        return(1-ES(0.05, rh)/ES(0.05, rs))

    elif rm == 'VaR q=0.01':
        return(1-VaR(0.01, rh)/VaR(0.01, rs))

    elif rm == 'VaR q=0.05':
        return(1-VaR(0.05, rh)/VaR(0.05, rs))

    elif rm == 'Variance':
        return(1-Variance(rh)/Variance(rs))

In [13]:
def rename_rm(rm):
    rm = rm.replace('q=','')
    rm = rm.replace('0.01', '99%')
    rm = rm.replace('0.05', '95%')
    return rm

In [14]:
config['spot_name']

'log return CRIX'

In [15]:
for i in range(len(OHR_to_use)):
    try:
        file = OHR_to_use.iloc[i,:].file_name
        rs = pd.read_csv(v5_path+file).loc[:,config['spot_name']]
        rf = pd.read_csv(v5_path+file).loc[:,config['future_name']]
        h = OHR_to_use.iloc[i, :].OHR
        rm = OHR_to_use.iloc[i,:].risk_measure
        OHR_to_use.iloc[i,-1] = hedging_effectiveness2(rm, rs, rf, h)
    except:
        OHR_to_use.iloc[i,-1] = np.NAN
        print(file +' is not in ls')
        continue
OHR_to_use.dropna(inplace=True)

In [16]:
i_id = OHR_to_use.risk_measure == rm
j_id = OHR_to_use.copula == C

In [17]:
path = '../latex/_pics/CRIX_future_Open_hedging_effectiveness_100days_with_v3_OHR/'

if os.path.exists(path) == False:
    os.mkdir(path)
    
for rm in risk_measure:
    i_id = OHR_to_use.risk_measure == rm
    
    plt.figure(figsize=(10,5))
    plt.title('Out-of-Sample Hedging Effectiveness of %s' %rename_rm(rm))
    
    for j, C in enumerate(Copula_names):
        j_id = OHR_to_use.copula == C
        plt.boxplot(OHR_to_use.loc[i_id&j_id,'HE'], positions=[j])
    plt.ylim((0,1))
    plt.xticks(range(len(Copula_names)), Copula_names)
    plt.savefig(path+'%s_100days_with_v3_OHR.pdf'%(rename_rm(rm).replace(' ', '_')))
    plt.close()

In [18]:
import itertools
a = ['HR 1']
b = risk_measure.copy()
c = ls.copy()
d = [1]
naive = pd.DataFrame(list(itertools.product(*[a,b,c,d])))
naive.columns = ['copula', 'risk_measure', 'file_name', 'OHR']
naive.loc[:,'HE'] = 0

In [19]:
OHR_to_use = OHR_to_use.append(naive)

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [21]:
for i in range(len(OHR_to_use)):
    try:
        file = OHR_to_use.iloc[i,:].file_name
        rs = pd.read_csv(v5_path+file).loc[:,config['spot_name']]
        rf = pd.read_csv(v5_path+file).loc[:,config['future_name']]
        h = OHR_to_use.iloc[i, :].OHR
        rm = OHR_to_use.iloc[i,:].risk_measure
        OHR_to_use.iloc[i,0] = hedging_effectiveness2(rm, rs, rf, h)
    except:
        OHR_to_use.iloc[i,-1] = np.NAN
        print(file +' is not in ls')
        continue
OHR_to_use.dropna(inplace=True)


In [22]:
path = '../latex/_pics/CRIX_future_hedging_effectiveness_100days_with_v3_OHR_with_naive/'

if os.path.exists(path) == False:
    os.mkdir(path)
    
for rm in risk_measure:
    i_id = OHR_to_use.risk_measure == rm
    
    plt.figure(figsize=(12,5))
    plt.title('Out-of-Sample Hedging Effectiveness of %s' %rename_rm(rm))
    
    for j, C in enumerate(Copula_names):
        j_id = OHR_to_use.copula == C
        plt.boxplot(OHR_to_use.loc[i_id&j_id,'HE'], positions=[j])
   
    j_id = OHR_to_use.copula == 'HR 1'
    plt.boxplot(OHR_to_use.loc[i_id&j_id,'HE'], positions=[j+1])
    plt.ylim((0,1))
    plt.xticks(range(len(Copula_names)+1), Copula_names+['HR 1'])
    plt.savefig(path+'%s_100days_with_v3_OHR_with_naive.pdf'%(rename_rm(rm).replace(' ', '_')))
    plt.close()